# Merging BAMs from different batches

In [2]:
#%%bash
module load samtools sambamba

WORKDIR=~/snap_hap
bamDIR=$WORKDIR/bams

# ## Make sample lists
# ls -hl $bamDIR/v3.5/ema_align/10xNEW-Plate10/*bam | cut -d_ -f6 | sort > $bamDIR/samples/samples_10xNEW-Plate10.txt
# ls -hl $bamDIR/v3.5/ema_align/10xNEW-Plate9/*bam | cut -d_ -f6 | sort > $bamDIR/samples/samples_10xNEW-Plate9.txt

In [3]:
## 10xNEW-Plate9
plate9=$bamDIR/samples/samples_10xNEW-Plate9.txt
wc -l $plate9
#head $plate9

64 /nfs/scistore18/bartogrp/apal/snap_hap/bams/samples/samples_10xNEW-Plate9.txt


In [4]:
## 10xNEW-Plate10
plate10=$bamDIR/samples/samples_10xNEW-Plate10.txt
wc -l $plate10
#head $plate10

96 /nfs/scistore18/bartogrp/apal/snap_hap/bams/samples/samples_10xNEW-Plate10.txt


In [5]:
## Make combined sample list for all 960 plants
# cat ~/snap_hap/sample_info/sample_MEGA_n960_2022/samples_{2x,10x,60x}_* > ~/snap_hap/sample_info/sample_MEGA_n960_2022/samples_ALL_n960.txt
wc -l ~/snap_hap/sample_info/sample_MEGA_n960_2022/samples_ALL_n960.txt
samples=~/snap_hap/sample_info/sample_MEGA_n960_2022/samples_ALL_n960.txt

960 /nfs/scistore18/bartogrp/apal/snap_hap/sample_info/sample_MEGA_n960_2022/samples_ALL_n960.txt


In [6]:
## Make bamlist to merge

# while read plantID
# do
#     #echo $plantID
#     paste <(echo $plantID) \
#             <(grep $plantID <(ls ~/snap_hap/bams/v3.5/ema_align/{2x*-*,10x,10xE,10xNEW*,60x,60xE}/*bam)) |\
#             paste - - - - >> ~/snap_hap/bams/v3.5/sample_list/bams_to_merge.txt
# done < $samples

In [7]:
wc -l ~/snap_hap/bams/v3.5/sample_list/bams_to_merge.txt

960 /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/sample_list/bams_to_merge.txt


## Merge 1: ID - pb2320  (10xNEW-Plate10 + 2xE-N710e + 2x-N710)

In [10]:
## Set variables
merged_bamDIR=/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023
bams_to_merge=/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/sample_list/bams_to_merge.txt

sed -n "900p" $bams_to_merge

pb2320	/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/10xNEW-Plate10/10xNEW-Plate10-57_Am_Pla_pb2320_v3.5.sorted.BXnum.bam		/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/2xE-N710e/2xE-N710e-53_Am_Pla_pb2320_v3.5.sorted.BXnum.bam		/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/2x-N710/2x-N710-53_Am_Pla_pb2320_v3.5.sorted.BXnum.bam	


In [11]:
## Read plantID and bamlist
read plantID bamlist < <(sed -n "900p" $bams_to_merge)

if [ $(echo $bamlist | tr " " "\n" | wc -l ) = 3 ];
then
    tmp=$(basename $(echo $bamlist | tr " " "\n" | head -1))
    outBAM=${tmp/.bam/.merged.bam}
else
    tmp=$(basename $(echo $bamlist | tr " " "\n" | tail -1))
    outBAM=${tmp/.bam/.merged.bam}
fi

echo PlantID: $plantID
echo BAMlist: $bamlist
echo outBAM: $outBAM

PlantID: pb2320
BAMlist: /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/10xNEW-Plate10/10xNEW-Plate10-57_Am_Pla_pb2320_v3.5.sorted.BXnum.bam /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/2xE-N710e/2xE-N710e-53_Am_Pla_pb2320_v3.5.sorted.BXnum.bam /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/2x-N710/2x-N710-53_Am_Pla_pb2320_v3.5.sorted.BXnum.bam
outBAM: 10xNEW-Plate10-57_Am_Pla_pb2320_v3.5.sorted.BXnum.merged.bam


In [80]:
## Merge BAMs with samtools
time samtools merge -@20 --no-PG -fcp -o ${merged_bamDIR}/$outBAM $bamlist


real	0m38.956s
user	4m22.046s
sys	0m3.086s


In [76]:
## Merge BAMs with sambamba
time sambamba merge -t20 -p ${merged_bamDIR}/$outBAM $bamlist


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)

[==============================================================================]

real	0m36.058s
user	8m30.959s
sys	0m17.471s


In [88]:
# Extract relevant header from BAM
samtools view -H ${merged_bamDIR}/$outBAM | grep -Ev '@PG|@RG'  > ${merged_bamDIR}/RG_$plantID.txt

# Add read group (@RG)
read_group_ID=${plantID}_$(echo $bamlist | cut -d/ -f10,20,30 | tr "/" "_")
read_group_SM=${tmp/.sorted.BXnum.bam}
read_group_LB=Snapdragon-Haplo_${tmp/_*}
read_group=@RG\\tID:${read_group_ID}\\tSM:${read_group_SM}\\tLB:${read_group_LB}
echo -e $read_group >> ${merged_bamDIR}/RG_${plantID}.txt

cat ${merged_bamDIR}/RG_${plantID}.txt

@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN:Chr8	LN:57431585
@RG	ID:pb2320_10xNEW-Plate10_2xE-N710e_2x-N710	SM:10xNEW-Plate10-57_Am_Pla_pb2320_v3.5	LB:Snapdragon-Haplo_10xNEW-Plate10-57


In [90]:
# Fix BAM header
time samtools reheader ${merged_bamDIR}/RG_${plantID}.txt ${merged_bamDIR}/$outBAM > ${merged_bamDIR}/tmp_$plantID.bam


real	0m2.046s
user	0m0.120s
sys	0m1.777s


In [91]:
# Rename the re-headered file with outBAM file
mv ${merged_bamDIR}/tmp_$plantID.bam ${merged_bamDIR}/$outBAM

# Remove header file
rm ${merged_bamDIR}/RG_${plantID}.txt

# Index merged BAM file
samtools index -@20 ${merged_bamDIR}/$outBAM


real	0m4.809s
user	0m22.903s
sys	0m2.657s


In [92]:
# Check the new BAM file
samtools view -H ${merged_bamDIR}/$outBAM

@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN:Chr8	LN:57431585
@RG	ID:pb2320_10xNEW-Plate10_2xE-N710e_2x-N710	SM:10xNEW-Plate10-57_Am_Pla_pb2320_v3.5	LB:Snapdragon-Haplo_10xNEW-Plate10-57
@PG	ID:samtools	PN:samtools	VN:1.16	CL:samtools reheader /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/RG_pb2320.txt /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xNEW-Plate10-57_Am_Pla_pb2320_v3.5.sorted.BXnum.merged.bam
@PG	ID:samtools.1	PN:samtools	PP:samtools	VN:1.16	CL:samtools view -H /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xNEW-Plate10-57_Am_Pla_pb2320_v3.5.sorted.BXnum.merged.bam


In [93]:
ls -hl ${merged_bamDIR}

total 1.5G
-rw-r--r-- 1 apal bartogrp 1.7G Jul  6 00:14 10xNEW-Plate10-57_Am_Pla_pb2320_v3.5.sorted.BXnum.merged.bam
-rw-r--r-- 1 apal bartogrp 476K Jul  6 00:14 10xNEW-Plate10-57_Am_Pla_pb2320_v3.5.sorted.BXnum.merged.bam.bai


## Merge 2: ID - pb2320  (10xNEW-Plate10 + 2xE-N710e + 2x-N710)

In [12]:
## Set variables
merged_bamDIR=/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023
bams_to_merge=/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/sample_list/bams_to_merge.txt

sed -n "100p" $bams_to_merge

pb0005	/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/2xE-N702e/2xE-N702e-5_Am_Pla_pb0005_v3.5.sorted.BXnum.bam		/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/2x-N702/2x-N702-5_Am_Pla_pb0005_v3.5.sorted.BXnum.bam		


In [14]:
## Read plantID and bamlist
read plantID bamlist < <(sed -n "100p" $bams_to_merge)

if [ $(echo $bamlist | tr " " "\n" | wc -l ) = 3 ];
then
    tmp=$(basename $(echo $bamlist | tr " " "\n" | head -1))
    outBAM=${tmp/.bam/.merged.bam}
else
    tmp=$(basename $(echo $bamlist | tr " " "\n" | tail -1))
    outBAM=${tmp/.bam/.merged.bam}
fi

echo PlantID: $plantID
echo BAMlist: $bamlist
echo outBAM: $outBAM

PlantID: pb0005
BAMlist: /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/2xE-N702e/2xE-N702e-5_Am_Pla_pb0005_v3.5.sorted.BXnum.bam /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/ema_align/2x-N702/2x-N702-5_Am_Pla_pb0005_v3.5.sorted.BXnum.bam
outBAM: 2x-N702-5_Am_Pla_pb0005_v3.5.sorted.BXnum.merged.bam


In [15]:
## Merge BAMs with samtools
time samtools merge -@20 --no-PG -fcp -o ${merged_bamDIR}/$outBAM $bamlist


real	0m14.601s
user	1m31.088s
sys	0m1.351s


In [16]:
## Merge BAMs with sambamba
time sambamba merge -t20 -p ${merged_bamDIR}/$outBAM $bamlist


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)

[==============================================================================]

real	0m13.775s
user	3m1.992s
sys	0m6.056s


In [17]:
# Extract relevant header from BAM
samtools view -H ${merged_bamDIR}/$outBAM | grep -Ev '@PG|@RG'  > ${merged_bamDIR}/RG_$plantID.txt

# Add read group (@RG)
read_group_ID=${plantID}_$(echo $bamlist | cut -d/ -f10,20,30 | tr "/" "_")
read_group_SM=${tmp/.sorted.BXnum.bam}
read_group_LB=Snapdragon-Haplo_${tmp/_*}
read_group=@RG\\tID:${read_group_ID}\\tSM:${read_group_SM}\\tLB:${read_group_LB}
echo -e $read_group >> ${merged_bamDIR}/RG_${plantID}.txt

cat ${merged_bamDIR}/RG_${plantID}.txt

@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN:Chr8	LN:57431585
@RG	ID:pb0005_2xE-N702e_2x-N702	SM:2x-N702-5_Am_Pla_pb0005_v3.5	LB:Snapdragon-Haplo_2x-N702-5


In [18]:
# Fix BAM header
samtools reheader ${merged_bamDIR}/RG_${plantID}.txt ${merged_bamDIR}/$outBAM > ${merged_bamDIR}/tmp_$plantID.bam

In [19]:
# Rename the re-headered file with outBAM file
mv ${merged_bamDIR}/tmp_$plantID.bam ${merged_bamDIR}/$outBAM

# Remove header file
rm ${merged_bamDIR}/RG_${plantID}.txt

# Index merged BAM file
samtools index -@20 ${merged_bamDIR}/$outBAM

In [20]:
# Check the new BAM file
samtools view -H ${merged_bamDIR}/$outBAM

@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN:Chr8	LN:57431585
@RG	ID:pb0005_2xE-N702e_2x-N702	SM:2x-N702-5_Am_Pla_pb0005_v3.5	LB:Snapdragon-Haplo_2x-N702-5
@PG	ID:samtools	PN:samtools	VN:1.16	CL:samtools reheader /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/RG_pb0005.txt /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/2x-N702-5_Am_Pla_pb0005_v3.5.sorted.BXnum.merged.bam
@PG	ID:samtools.1	PN:samtools	PP:samtools	VN:1.16	CL:samtools view -H /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/2x-N702-5_Am_Pla_pb0005_v3.5.sorted.BXnum.merged.bam


In [21]:
ls -hl ${merged_bamDIR}

total 645M
-rw-r--r-- 1 apal bartogrp 701M Jul  6 11:16 2x-N702-5_Am_Pla_pb0005_v3.5.sorted.BXnum.merged.bam
-rw-r--r-- 1 apal bartogrp 395K Jul  6 11:16 2x-N702-5_Am_Pla_pb0005_v3.5.sorted.BXnum.merged.bam.bai


## Running SLURM job to merge BAMs

In [ ]:
cd /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/jobs/merge_bams_2023

sbatch --array=1-960 ~/snap_hap/_scripts/sbatch/bam-utils/job-merge_bams.sbatch

## Manually fixing problematic sample names

* 2 samples with 60x coverage are included in 10xNEW - named, 60x2
* 6 samples with 10x coverage are included in 10xNEW - named, 10x2
* mislabelled as 10xE instead of 10x
* 1 samlple: mislabelled as 60xE instead of 60x

### 1. 60x2 samples (*n = 2*)

In [1]:
bamINFO=~/snap_hap/sample_info/bam_info/samples_mergedBAMs_2023.txt
cat $bamINFO | grep 60x2

pb1042	60xE	60x	10xNEW-Plate9	60x2
pb0898	60xE	60x	10xNEW-Plate10	60x2


In [2]:
## Fix header
for plantID in pb1042 pb0898
do
    echo -e "Fixing PlantID $plantID\n"
    # bash ~/snap_hap/_scripts/bash/reheader_bams.sh $plantID 10xNEW 60x2
    echo -e "\n"
done

Fixing PlantID pb1042

PlantID: pb1042
BAMfile: /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xNEW-Plate9-17_Am_Pla_pb1042_v3.5.sorted.BXnum.merged.bam


 New Header
@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN:Chr8	LN:57431585
@RG	ID:pb1042_10xNEW-Plate9_60x_60xE	SM:60x2-Plate9-17_Am_Pla_pb1042_v3.5	LB:Snapdragon-Haplo_60x2-Plate9-17


 Fixing BAM Header

real	0m49.511s
user	0m1.806s
sys	0m28.682s


 Indexing BAM
[E::hts_open_format] Failed to open file "/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xNEW-Plate9-17_Am_Pla_pb1042_v3.5.sorted.BXnum.merged.bam" : No such file or directory
samtools index: failed to open "/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xNEW-Plate9-17_Am_Pla_pb1042_v3.5.sorted.BXnum.merged.bam": No such file or directory

real	0m0.022s
user	0m

### 2. 10x2 samples (*n = 6*)

In [4]:
bamINFO=~/snap_hap/sample_info/bam_info/samples_mergedBAMs_2023.txt
cat $bamINFO | grep 10x2

pb1196	10xE	10x	10xNEW-Plate9	10x2
pb1008	10xE	10x	10xNEW-Plate9	10x2
pb1253	10xE	10x	10xNEW-Plate9	10x2
pb1682	10xE	10x	10xNEW-Plate10	10x2
pb2190	10xE	10x	10xNEW-Plate10	10x2
pb1687	10xE	10x	10xNEW-Plate10	10x2


In [20]:
IDs=$(cat $bamINFO | grep 10x2 | cut -f1)
echo $IDs

pb1196 pb1008 pb1253 pb1682 pb2190 pb1687


In [21]:
IDs=$(cat $bamINFO | grep 10x2 | cut -f1)
for plantID in $IDs
do
    echo -e "Fixing PlantID $plantID\n"
    bash ~/snap_hap/_scripts/bash/reheader_bams.sh $plantID 10x 10x2
    echo -e "\n"
done

Fixing PlantID pb1196

PlantID: pb1196
BAMfile: /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10x-13_Am_Pla_pb1196_v3.5.sorted.BXnum.merged.bam


 New Header
@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN:Chr8	LN:57431585
@RG	ID:pb1196_10x_10xE_10xNEW-Plate9	SM:10x2-13_Am_Pla_pb1196_v3.5	LB:Snapdragon-Haplo_10x2-13


 Fixing BAM Header

real	0m11.716s
user	0m0.485s
sys	0m7.730s


 Indexing BAM

real	0m44.636s
user	0m41.416s
sys	0m2.699s

Done!


Fixing PlantID pb1008

PlantID: pb1008
BAMfile: /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10x-14_Am_Pla_pb1008_v3.5.sorted.BXnum.merged.bam


 New Header
@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN

### 3. 10x samples mislabelled as 10xE (n=)

In [25]:
bamINFO=~/snap_hap/sample_info/bam_info/samples_mergedBAMs_2023.txt
cat $bamINFO | grep 10xE

pb0174	10xE	10x	NaN	10x
pb0216	10xE	10x	NaN	10x
pb0294	10xE	10x	NaN	10x
pb0349	10xE	10x	NaN	10x
pb0403	10xE	10x	NaN	10x
pb0458	10xE	10x	NaN	10x
pb0407	10xE	10x	NaN	10x
pb0768	10xE	10x	NaN	10x
pb2333	10xE	10x	NaN	10x
pb3403	10xE	10x	NaN	10x
pb4318	10xE	10x	NaN	10x
pb4563	10xE	10x	NaN	10x
pb1196	10xE	10x	10xNEW-Plate9	10x2
pb1008	10xE	10x	10xNEW-Plate9	10x2
pb1253	10xE	10x	10xNEW-Plate9	10x2
pb1682	10xE	10x	10xNEW-Plate10	10x2
pb2190	10xE	10x	10xNEW-Plate10	10x2
pb1687	10xE	10x	10xNEW-Plate10	10x2


In [28]:
ls ~/snap_hap/bams/v3.5/bams_merged_2023/*10xE*bam | wc -l
ls ~/snap_hap/bams/v3.5/bams_merged_2023/*10xE*bam

12
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-10_Am_Pla_pb3403_v3.5.sorted.BXnum.merged.bam
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-11_Am_Pla_pb4318_v3.5.sorted.BXnum.merged.bam
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-12_Am_Pla_pb4563_v3.5.sorted.BXnum.merged.bam
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-1_Am_Pla_pb0174_v3.5.sorted.BXnum.merged.bam
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-2_Am_Pla_pb0216_v3.5.sorted.BXnum.merged.bam
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-3_Am_Pla_pb0294_v3.5.sorted.BXnum.merged.bam
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-4_Am_Pla_pb0349_v3.5.sorted.BXnum.merged.bam
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-5_Am_Pla_pb0403_v3.5.sorted.BXnum.merged.bam
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/

In [26]:
IDs=$(cat $bamINFO | grep 10xE | head -12 | cut -f1)
echo $IDs

pb0174 pb0216 pb0294 pb0349 pb0403 pb0458 pb0407 pb0768 pb2333 pb3403 pb4318 pb4563


In [29]:
IDs=$(cat $bamINFO | grep 10xE | head -12 | cut -f1)
for plantID in $IDs
do
    echo -e "Fixing PlantID $plantID\n"
    bash ~/snap_hap/_scripts/bash/reheader_bams.sh $plantID 10xE 10x
    echo -e "\n"
done

Fixing PlantID pb0174

PlantID: pb0174
BAMfile: /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-1_Am_Pla_pb0174_v3.5.sorted.BXnum.merged.bam


 New Header
@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN:Chr8	LN:57431585
@RG	ID:pb0174_10x_10xE	SM:10x-1_Am_Pla_pb0174_v3.5	LB:Snapdragon-Haplo_10x-1


 Fixing BAM Header

real	0m13.910s
user	0m0.528s
sys	0m7.858s


 Indexing BAM

real	0m49.800s
user	0m44.233s
sys	0m2.931s

Done!


Fixing PlantID pb0216

PlantID: pb0216
BAMfile: /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/10xE-2_Am_Pla_pb0216_v3.5.sorted.BXnum.merged.bam


 New Header
@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN:Chr8	LN:57431585


### 4. 60x samples mislabelled as 60xE (*n = 1*)

In [22]:
bamINFO=~/snap_hap/sample_info/bam_info/samples_mergedBAMs_2023.txt
cat $bamINFO | grep 60xE

pb0464	60xE	60x	NaN	60x
pb1042	60xE	60x	10xNEW-Plate9	60x2
pb0898	60xE	60x	10xNEW-Plate10	60x2


Only **pb0464** is the relevant plantID to be changed. 

In [23]:
ls ~/snap_hap/bams/v3.5/bams_merged_2023/*pb0464*

/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/60xE-1_Am_Pla_pb0464_v3.5.sorted.BXnum.merged.bam
/nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/60xE-1_Am_Pla_pb0464_v3.5.sorted.BXnum.merged.bam.bai


In [24]:
for plantID in pb0464
do
    echo -e "Fixing PlantID $plantID\n"
    bash ~/snap_hap/_scripts/bash/reheader_bams.sh $plantID 60xE 60x
    echo -e "\n"
done

Fixing PlantID pb0464

PlantID: pb0464
BAMfile: /nfs/scistore18/bartogrp/apal/snap_hap/bams/v3.5/bams_merged_2023/60xE-1_Am_Pla_pb0464_v3.5.sorted.BXnum.merged.bam


 New Header
@HD	VN:1.3	SO:coordinate
@SQ	SN:Chr1	LN:71919034
@SQ	SN:Chr2	LN:77118269
@SQ	SN:Chr3	LN:65231163
@SQ	SN:Chr4	LN:54887108
@SQ	SN:Chr5	LN:71106538
@SQ	SN:Chr6	LN:55699338
@SQ	SN:Chr7	LN:55564713
@SQ	SN:Chr8	LN:57431585
@RG	ID:pb0464_60x_60xE	SM:60x-1_Am_Pla_pb0464_v3.5	LB:Snapdragon-Haplo_60x-1


 Fixing BAM Header

real	1m18.572s
user	0m2.525s
sys	0m37.499s


 Indexing BAM

real	4m15.485s
user	3m55.995s
sys	0m13.277s

Done!


